Individual NBA player points regression \
Plans
- gather data
    - save to a csv
- try a simple linear regression
- try more complex random forest regression
- use random forest to create confidence intervals

In [37]:
from nba_api.stats.endpoints import PlayerGameLogs
from nba_api.stats.static import players
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os

In [19]:
df = pd.read_csv("./data/2023_nba_player_stats.csv")

In [ ]:
# Get the player game logs with the correct parameter
player_game_logs = PlayerGameLogs(season_nullable='2023-24')
player_game_logs_df = player_game_logs.get_data_frames()[0]

# createa csv file with the player game logs
player_game_logs_df.to_csv("./data/logs_2023_nba_players.csv", index=False)

In [44]:
print(player_game_logs_df.columns)

Index(['SEASON_YEAR', 'PLAYER_ID', 'PLAYER_NAME', 'NICKNAME', 'TEAM_ID',
       'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID', 'GAME_DATE', 'MATCHUP',
       'WL', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM',
       'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK',
       'BLKA', 'PF', 'PFD', 'PTS', 'PLUS_MINUS', 'NBA_FANTASY_PTS', 'DD2',
       'TD3', 'WNBA_FANTASY_PTS', 'GP_RANK', 'W_RANK', 'L_RANK', 'W_PCT_RANK',
       'MIN_RANK', 'FGM_RANK', 'FGA_RANK', 'FG_PCT_RANK', 'FG3M_RANK',
       'FG3A_RANK', 'FG3_PCT_RANK', 'FTM_RANK', 'FTA_RANK', 'FT_PCT_RANK',
       'OREB_RANK', 'DREB_RANK', 'REB_RANK', 'AST_RANK', 'TOV_RANK',
       'STL_RANK', 'BLK_RANK', 'BLKA_RANK', 'PF_RANK', 'PFD_RANK', 'PTS_RANK',
       'PLUS_MINUS_RANK', 'NBA_FANTASY_PTS_RANK', 'DD2_RANK', 'TD3_RANK',
       'WNBA_FANTASY_PTS_RANK', 'AVAILABLE_FLAG', 'MIN_SEC'],
      dtype='object')


In [ ]:
def get_player_data(player_name, season='2023-24'):
    # Get player ID
    player_dict = players.find_players_by_full_name(player_name)[0]
    player_id = player_dict['id']
    print(player_id)
    
    # Get game logs
    game_logs = PlayerGameLogs(player_id=player_id, season=season).get_data_frames()[0]
    
    # Process into required features
    df = pd.DataFrame()
    
    # Basic game info
    df['game_date'] = pd.to_datetime(game_logs['GAME_DATE'])
    df['points'] = game_logs['PTS']
    df['minutes'] = game_logs['MIN']
    df['is_home'] = game_logs['MATCHUP'].str.contains('vs.')
    
    # Calculate rolling averages
    df['points_ma'] = df['points'].rolling(5, min_periods=1).mean()
    df['minutes_played_ma'] = df['minutes'].rolling(5, min_periods=1).mean()
    
    # Calculate rest days and back-to-backs
    df['days_rest'] = df['game_date'].diff().dt.days
    df['back_to_back'] = df['days_rest'] == 1
    
    # Season averages
    df['season_avg_points'] = df['points'].expanding().mean()
    
    # Last 5 games points (as a list)
    df['last_5_games_points'] = df['points'].rolling(5, min_periods=1).apply(list)
    
    return df

# Example usage
player_data = get_player_data('Stephen Curry')

201939


TypeError: PlayerGameLogs.__init__() got an unexpected keyword argument 'player_ids'